In [1]:
import pickle as pkl
import numpy as np

In [2]:
X0, X1, Y, vocab_to_ind = pkl.load(open('../data/data.npy', 'rb'))

In [34]:
vocab_to_ind[' '] = 29

In [35]:
pkl.dump(vocab_to_ind, open('word_to_idx.pkl', 'wb'))

In [3]:
Y0 = np.eye(X0.shape[1])
Y0 = np.concatenate([Y0, np.zeros((2, X0.shape[1]))], axis=0)[np.newaxis,]
Y0 = np.tile(Y0, (len(X0),1,1))
Y0.shape

l0, l1 = X0.shape[1], X1.shape[1]
B = np.tile(np.arange(l0).reshape((1,-1)), (l1, 1)) 
B = -1e9 * (B < np.arange(l1)[:,np.newaxis]).astype(int)
B = np.tile(B[np.newaxis,], (len(X0),1,1))

In [5]:
import models
mask_meta = '<m>'
model = models.pointer_transformer(alphabet_size=len(vocab_to_ind)+1,
                                   blocks=2,
                                   num_heads=4,
                                   dim=64,
                                   dropout=None,
                                   mask_meta=vocab_to_ind[mask_meta])

/usr/lib/python3/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/lib/python3/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/lib/python3/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/lib/python3/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is 

In [6]:
import tensorflow as tf
from tensorflow import keras

def loss(y_true, y_pred):
    mask = tf.cast(tf.not_equal(y_true[:, :, 2:3], 1.), tf.float32)
    logits = tf.log(y_pred + 1e-9)
    return - mask * y_true * logits

In [7]:
inds = np.arange(len(Y0))
np.random.shuffle(inds)

Ntr = int(.7*len(Y0))
train_inds = inds[:Ntr]
test_inds = inds[Ntr:]

In [8]:
#_h = history

In [10]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

rlop = ReduceLROnPlateau(monitor='loss', verbose=1, factor=.5, patience=5)
es = EarlyStopping(monitor='loss', verbose=1, patience=12)
mc = ModelCheckpoint('ANN_weights_s3.h5', verbose=1, monitor='val_loss', save_best_only=True)

model.compile(keras.optimizers.Adam(1e-3), loss)
history = model.fit([X1[train_inds], X0[train_inds], B[train_inds]], Y0[train_inds],
                    batch_size=1024, epochs=150,
                    validation_data=([X1[test_inds], X0[test_inds], B[test_inds]], Y0[test_inds]),
                    callbacks=[rlop, es, mc])

Train on 230603 samples, validate on 98830 samples
Epoch 1/150
230400/230603 [============================>.] - ETA: 0s - loss: 0.0681
Epoch 00001: val_loss improved from inf to 0.06101, saving model to ANN_weights_s3.h5
230603/230603 [==============================] - 17s 76us/sample - loss: 0.0681 - val_loss: 0.0610
Epoch 2/150
230400/230603 [============================>.] - ETA: 0s - loss: 0.0590
Epoch 00002: val_loss improved from 0.06101 to 0.05808, saving model to ANN_weights_s3.h5
230603/230603 [==============================] - 11s 49us/sample - loss: 0.0590 - val_loss: 0.0581
Epoch 3/150
230400/230603 [============================>.] - ETA: 0s - loss: 0.0572
Epoch 00003: val_loss improved from 0.05808 to 0.05691, saving model to ANN_weights_s3.h5
230603/230603 [==============================] - 11s 48us/sample - loss: 0.0572 - val_loss: 0.0569
Epoch 4/150
230400/230603 [============================>.] - ETA: 0s - loss: 0.0561
Epoch 00004: val_loss improved from 0.05691 to 0.0

KeyboardInterrupt: 

In [11]:
model.load_weights('ANN_weights_s3.h5')

In [15]:
start_meta = '<s>'
end_meta = '<e>'

start = [start_meta]
s0 = np.array([vocab_to_ind[c] for c in start]).reshape((1,-1))

letters = list('ghost')
letter_inds = np.array([vocab_to_ind[c] for c in letters]).reshape((1,-1))

In [16]:
from utils import 

ind_to_vocab = {v:k for k,v in  vocab_to_ind.items()}
def write_out(s, ind_to_vocab):
    string = ''
    for n in s[0]:
        string += ind_to_vocab[n]
    return string

for _ in range(len(letters)):
    nxt_p = model.predict([s0, letter_inds])[0,-1]
    nxt_n = np.random.choice(len(nxt_p), p=nxt_p) # nxt_p.argmax()
    nxt_c = letters[nxt_n]
    s0 = np.concatenate([s0, vocab_to_ind[nxt_c] * np.ones((1,1))], -1)
    print(write_out(s0))

<s>g
<s>go
<s>got
<s>goth
<s>goths


In [58]:
import tensorflow as tf

z = tf.expand_dims(tf.range(6)[::-1], -1)
z = tf.tile(z, (1,4))

x0 = tf.constant([[2,1,2,3]])
x1 = tf.constant([[0,2,2,3,1,4]])

chk_bin = tf.equal(tf.transpose(x1), x0)
chk_bin = tf.cast(chk_bin, tf.int32)
chk = (z+1)*chk_bin

bal = tf.expand_dims(1+tf.range(4), 0)
chk_bal = bal*chk_bin
#chk_bal = tf.reduce_max(chk_bal, -1)
chk_bal = tf.greater_equal(chk_bal, tf.reduce_max(chk_bal, -1, keepdims=True))
chk_bal = chk_bin*tf.cast(chk_bal, tf.int32)

chk = tf.less_equal(z+1, tf.reduce_max(chk, 0))
chk = tf.cast(chk, tf.int32)

with tf.Session() as sess:
    print(sess.run(z+1))
    print(sess.run(chk))
    print(sess.run(chk_bal))

[[6 6 6 6]
 [5 5 5 5]
 [4 4 4 4]
 [3 3 3 3]
 [2 2 2 2]
 [1 1 1 1]]
[[0 0 0 0]
 [1 0 1 0]
 [1 0 1 0]
 [1 0 1 1]
 [1 1 1 1]
 [1 1 1 1]]
[[0 0 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 1 0 0]
 [0 0 0 0]]


In [32]:
import importlib
import utils
importlib.reload(utils)

ind_to_vocab = {v:k for k,v in vocab_to_ind.items()}
letters = list('function')
letter_inds = np.array([vocab_to_ind[c] for c in letters]).reshape((1,-1))
utils.beam_search(letter_inds, model.predict, vocab_to_ind, beam_length=5)

f : p=[0.32234677]
u : p=[0.27813451]
c : p=[0.25998696]
i : p=[0.07820435]
n : p=[0.06132741]

fu : p=[0.34347665]
co : p=[0.27862752]
un : p=[0.15545223]
un : p=[0.15545223]
no : p=[0.06699137]

cou : p=[0.30867757]
fun : p=[0.2180532]
fun : p=[0.2180532]
unf : p=[0.12760802]
unf : p=[0.12760801]

func : p=[0.24791789]
func : p=[0.24791789]
coun : p=[0.17981615]
coun : p=[0.17981615]
unfo : p=[0.14453193]

funct : p=[0.2651667]
funct : p=[0.26516667]
counf : p=[0.1679193]
counf : p=[0.1679193]
unfoc : p=[0.13382802]

functi : p=[0.28327749]
functi : p=[0.28327742]
counfi : p=[0.17864285]
counfi : p=[0.17864285]
unfoci : p=[0.07615938]

functio : p=[0.28309446]
functio : p=[0.28309439]
counfin : p=[0.17879373]
counfin : p=[0.17879373]
unfocin : p=[0.07622369]

function : p=[0.28309446]
function : p=[0.28309439]
counfint : p=[0.17879373]
counfint : p=[0.17879373]
unfocint : p=[0.07622369]



In [105]:
chk.shape

(1, 1, 6)

In [20]:
chk[0,-1]

array([0.22599305, 0.21628883, 0.03875758, 0.04105627, 0.42614967,
       0.0517546 ], dtype=float32)